In [1]:
#匯入套件
import requests
import pandas as pd
import json
import re
from lxml import etree
import time

#函式部分
BRAND = "順風醫美診所"
WEBSITE_URL = "https://www.sfclinic.com.tw/"
###爬取醫生info
def get_doctor_detail(doctor_url):
    resp = requests.get(doctor_url)
    root = etree.HTML(resp.content.decode('utf-8'))
    doctor_result = {
        "brand":BRAND,  
        "pic":WEBSITE_URL+"".join(root.xpath("//div[@class='box']//img/@data-lazy")), 
        "doctor_name":"".join(root.xpath("//h2[@class='ti']/span/text()")),
        "store_name":root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'服務館別')]/following-sibling::div/text()"),
        "personal_website":doctor_url,
        "specialty":root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'專長')]/following-sibling::div/text()"),
        "description":"",
        "case_study":"",
        "city":"",
        "address":"",
        "tel":"",
        "store_website":"",
        "experience":root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'經歷')]/following-sibling::div/text()"),
        "education":root.xpath("//div[@class='more-info']//div[@class='item'][contains(text(),'學歷')]/following-sibling::div/text()")
    }
    return doctor_result
###爬取醫生
def get_doctors():
    resp = requests.get( f"{WEBSITE_URL}/team.php")
    root = etree.HTML(resp.content.decode('utf-8'))
    doctor_urls = root.xpath("//div[contains(@class,'list-box')]/div/a/@href")
    return list(map(lambda i: f"{WEBSITE_URL}/{i}", doctor_urls))
###爬取診所info
def get_clinic_detail(clinic_url):
    resp = requests.get(clinic_url)
    root = etree.HTML(resp.content.decode('utf-8'))
    clinic_result = {
        "city":root.xpath("//h2[@class='store-name']/span/text()"),
        "address":root.xpath("//div[@class='address list']//i/following-sibling::text()"),
        "tel":root.xpath("//div[@class='tel list']//i/following-sibling::text()"),
        "store_website":clinic_url
    }
    return clinic_result
###爬取診所
def get_clinics():
    resp = requests.get( f"{WEBSITE_URL}/location.php")
    root = etree.HTML(resp.content.decode('utf-8'))
    clinic_urls =root.xpath("//div[contains(@class,'list-box')]/div/a/@href")
    return list(map(lambda i: f"{WEBSITE_URL}/{i}",clinic_urls))         
###診所資料﹑診所合併
clinic_list = get_clinics()
clinic_details = []
for clinic_idx,clinic_url in enumerate(clinic_list):
    print(f"{clinic_idx+1}/{len(clinic_list)}, {clinic_url}")
    clinic_details.append(get_clinic_detail(clinic_url))
##刪除不必要資訊
for i in range(len(clinic_details)):
    for i_city in range(len(clinic_details[i]['city'])):
        clinic_details[i]['city']=[sub.replace('店', '') for sub in clinic_details[i]['city']]
###醫生資料﹑醫生合併        
doctor_list = get_doctors()
doctor_details = []
for doctor_idx,doctor_url in enumerate(doctor_list):
    print(f"{doctor_idx+1}/{len(doctor_list)}, {doctor_url}")
    doctor_details.append(get_doctor_detail(doctor_url))
##資料分割    
def clean():
    for n in range(len(doctor_details)):
        doctor_details[n]['store_name']=",".join(doctor_details[n]['store_name'])
        doctor_details[n]['city']=",".join(doctor_details[n]['city'])
        doctor_details[n]['address']=",".join(doctor_details[n]['address'])
        doctor_details[n]['tel']="".join(doctor_details[n]['tel'])
        doctor_details[n]['specialty']=",".join(doctor_details[n]['specialty'])
        doctor_details[n]['experience']=",".join(doctor_details[n]['experience'])
        doctor_details[n]['education']=",".join(doctor_details[n]['education'])
    return doctor_details

#執行程式
FILE_NAME = BRAND
today = time.strftime("%Y%m%d")
df = pd.DataFrame(clean(), columns=["brand","pic","doctor_name","store_name","city","address","tel","personal_website","store_website","specialty","description","case_study","experience","education"])
df.to_csv(f"{FILE_NAME}-{today}.csv",index=False)